In [50]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

from transformers import AutoTokenizer
import transformers
import torch
import numpy as np
import scipy as sp
import shap

from utils import dump_jsonl, load_jsonl, set_random_seed
import torch
from transformers import AutoModelForSequenceClassification, AdamW, BertConfig

In [51]:
import pickle

def load_shap_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

def save_shap_values(filepath, obj):
    with open(filepath, 'wb') as fin:
        pickle.dump(obj, fin)

In [52]:
import pandas as pd

In [53]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 5

In [54]:
class ClassifierPredictor:
    def __init__(self, model, tokenizer, device, max_length=256):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_length = max_length


    # define a prediction function
    def predict(self, texts):
        tokenizer = self.tokenizer
        model = self.model
        device = self.device
        max_length = self.max_length

        text_ids = [tokenizer.encode(text, max_length=max_length, padding='max_length', truncation=True) for text in texts]

        att_masks = []
        for ids in text_ids:
                masks = [int(id > 0) for id in ids]
                att_masks.append(masks)

        text_ids = torch.tensor(text_ids).to(device)
        att_masks = torch.tensor(att_masks).to(device)

        outputs = model(text_ids, attention_mask=att_masks)
        outputs = outputs[0].detach().cpu().numpy()
        scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        
        val = sp.special.logit(scores[:,1]) # use one vs rest logit units
        return val

In [55]:
class RegressorPredictor:
    def __init__(self, model, tokenizer, device, label_fn, max_length=256):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.max_length = max_length
        self.label_fn = label_fn


    # define a prediction function
    def predict(self, texts):
        tokenizer = self.tokenizer
        model = self.model
        device = self.device
        max_length = self.max_length

        text_ids = [tokenizer.encode(text, max_length=max_length, padding='max_length', truncation=True) for text in texts]

        att_masks = []
        for ids in text_ids:
                masks = [int(id > 0) for id in ids]
                att_masks.append(masks)
#         print(len(texts))
        text_ids = torch.tensor(text_ids).to(device)
        att_masks = torch.tensor(att_masks).to(device)

        outputs = model(text_ids, attention_mask=att_masks)
        outputs = outputs[0].detach().cpu().numpy()
#         print(outputs)
#         print(text_ids.shape)
#         print("OUTPUT", outputs.shape)
        return outputs.reshape(-1)

In [56]:

    
def get_shapley(model_path, df, regressor=False, regressor_label_fn=None):
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    if regressor:
        p = RegressorPredictor(model, tokenizer, device, regressor_label_fn, max_length=max_length)
    else:
        p = ClassifierPredictor(model, tokenizer, device, max_length=max_length)
        
    texts = [preprocess(t) for t in df["text"].values]

    d = {"text": texts}
    explainer = shap.Explainer(p.predict, tokenizer)
    shap_values = explainer(d, fixed_context=1, batch_size=batch_size)
    return shap_values

In [57]:
# # shap_values[0, 0, 0]
# shap.plots.text(shap_values[0, :])

In [58]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

In [59]:
# _tmp = shap_values[:, :]
# shap_data = _tmp.data
# shap_values = _tmp.values

def get_shap_lexicons(df, shap_values, fn_abs=True):
    lexicons = {}
    label_values = df["label"].unique()
    

    for _, label in enumerate(label_values):
        d = df[df["label"]==label]
        if fn_abs:
            s = shap_values.abs[d.index.to_numpy()]
        else:
            s = shap_values[d.index.to_numpy()]

        feature_names = s.mean(0).feature_names
        shapley_values = s.mean(0).values

        sorted_values = sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0])
        lexicons[label] = {x.lower():v for v, x in sorted_values}
        print(label, "done")
    return lexicons

In [60]:
from collections import defaultdict

In [61]:
from pythainlp.tokenize import word_tokenize
import itertools
from tqdm import tqdm

def merge_subwords(df, lexicons, tokenizer):
    tf = defaultdict(int)
    newlexicons = {}
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        label = row["label"]
        if label not in newlexicons:
            newlexicons[label] = {}

        words = word_tokenize(preprocess(row["text"]))
        words = [w for w in words if len(w.strip())>0]
        
        tokenized_input = tokenizer(words, is_split_into_words=True)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
        word_ids = tokenized_input.word_ids(batch_index=0)

        tidx = 0
        for widx, group in itertools.groupby(word_ids):
            val = 0
            for _ in list(group): 
                t = tokens[tidx].replace("▁", "")
                tidx += 1

                if t in lexicons[label]:
                    val += lexicons[label][t]

            if widx is None:
                continue

            w = words[widx]
            newlexicons[label][w] = val

            tf[w] += 1
    return newlexicons, tf

In [62]:
import numpy as np
def filter_lexicon(lexicons, tf, q=0.50):
    values = []
    for label in lexicons:
        for word in lexicons[label]:
            values.append(lexicons[label][word])


    threshold = np.quantile(values, q)

    filtered_lexicons = {}
    cc = 0
    for label in lexicons:
        filtered_lexicons[label] = {}
        for word in lexicons[label]:
            val = lexicons[label][word]
            # if val <= threshold:
            #     continue
            
            if tf[word] < 5:
                continue

            filtered_lexicons[label][word] = val
            cc += 1
    print("TOTAL", cc)
    return filtered_lexicons


In [63]:
def get_lexicons(df, model_path, shap_path, lex_path, regressor=False):
    train, val, test = df
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    df = pd.concat([train, test, val])
    shap_values = get_shapley(model_path, df, regressor=regressor)
    
#     shap_values = load_shap_values(shap_path)
#     assert(len(df)==len(shap_values))
    save_shap_values(shap_path, shap_values)
    
    lexicons = get_shap_lexicons(df, shap_values)
    newlexicons, tf = merge_subwords(df, lexicons, tokenizer)
    newlexicons = filter_lexicon(newlexicons, tf)
    dump_jsonl(lex_path, [newlexicons])
    return 

In [64]:
# df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
# get_lexicons(df, f'./Models/task1_clse_usr/best_model', f"./ShapleyValues/task1_clse.pkl", "lexicon_task1_clse.jsonl")

In [65]:
df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
get_lexicons(df, f'./Regressors/task1_clse_usr/best_model', f"./ShapleyValues/task1_clse_regressor.pkl", "lexicon_task1_clse.jsonl", regressor=True)

In [39]:
# df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
# get_lexicons(df, f'./Models/task1_auth_usr/best_model', f"./ShapleyValues/task1_auth.pkl", "lexicon_task1_auth.jsonl")

In [66]:
df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
get_lexicons(df, f'./Regressors/task1_auth_usr/best_model', f"./ShapleyValues/task1_auth_regressor.pkl", "lexicon_task1_auth.jsonl", regressor=True)

In [41]:
# df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
# get_lexicons(df, f'./Models/task2_clse_usr/best_model', f"./ShapleyValues/task2_clse.pkl", "lexicon_task2_clse.jsonl")

In [67]:
df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
get_lexicons(df, f'./Regressors/task2_clse_usr/best_model', f"./ShapleyValues/task2_clse_regressor.pkl", "lexicon_task2_clse.jsonl", regressor=True)

In [43]:
# df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
# get_lexicons(df, f'./Models/task2_auth_usr/best_model', f"./ShapleyValues/task2_auth.pkl", "lexicon_task2_auth.jsonl")

In [68]:
df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
get_lexicons(df, f'./Regressors/task2_auth_usr/best_model', f"./ShapleyValues/task2_auth_regressor.pkl", "lexicon_task2_auth.jsonl", regressor=True)

In [45]:
# df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
# get_lexicons(df, f'./Models/task3_clse_usr/best_model', f"./ShapleyValues/task3_clse.pkl", "lexicon_task3_clse.jsonl")

In [69]:
df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
get_lexicons(df, f'./Regressors/task3_clse_usr/best_model', f"./ShapleyValues/task3_clse_regressor.pkl", "lexicon_task3_clse.jsonl", regressor=True)

In [47]:
# df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
# get_lexicons(df, f'./Models/task3_auth_usr/best_model', f"./ShapleyValues/task3_auth.pkl", "lexicon_task3_auth.jsonl")

In [70]:
df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
get_lexicons(df, f'./Regressors/task3_auth_usr/best_model', f"./ShapleyValues/task3_auth_regressor.pkl", "lexicon_task3_auth.jsonl", regressor=True)